In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.neural_network import MLPClassifier
#from sklearn.model_selection import cross_val_score
#from sklearn.metrics import accuracy_score
#from sklearn.model_selection import GridSearchCV
#from sklearn.externals import joblib
#import xgboost as xgb


In [3]:
#import des donnees

path='C:\\Users\\oussa\\Downloads\\Data Science\\Master Statistique Big Data Dauphine\\Module 3\\Mini-Projet\\'
data_train=pd.read_csv(path+'input_train.csv')
train_output=pd.read_csv(path+'challenge_fichier_de_sortie_dentrainement_prevision_du_statut_des_reclamations_liees_aux_transactions.csv',sep=';')
data_train=pd.merge(data_train,train_output,left_on='ID',right_on='ID',how='inner')
data_train.head()


,ID,SHIPPING_MODE,SHIPPING_PRICE,WARRANTIES_FLG,WARRANTIES_PRICE,CARD_PAYMENT,COUPON_PAYMENT,RSP_PAYMENT,WALLET_PAYMENT,PRICECLUB_STATUS,...,BUYER_DEPARTMENT,BUYING_DATE,SELLER_SCORE_COUNT,SELLER_SCORE_AVERAGE,SELLER_COUNTRY,SELLER_DEPARTMENT,PRODUCT_TYPE,PRODUCT_FAMILY,ITEM_PRICE,CLAIM_TYPE
0,0,NORMAL,NaN,False,NaN,1,0,1,0,UNSUBSCRIBED,...,34,3/2017,10000<100000,46.0,"FRANCE, METROPOLITAN",61,CELLPHONE_ACCESSORY,ELECTRONICS,<10,DAMAGED
1,1,NORMAL,NaN,False,NaN,1,0,0,0,UNSUBSCRIBED,...,77,8/2017,10000<100000,45.0,"FRANCE, METROPOLITAN",30,CELLPHONE_ACCESSORY,ELECTRONICS,<10,-
2,2,NORMAL,NaN,False,NaN,0,0,0,1,PLATINUM,...,58,5/2017,10000<100000,43.0,CHINA,-1,TOYS,BABY,<10,NOT_RECEIVED
3,3,RECOMMANDE,NaN,True,5<20,1,0,0,0,UNSUBSCRIBED,...,31,5/2017,10000<100000,44.0,"FRANCE, METROPOLITAN",2,GARDEN_TOOLS,WHITE,50<100,-
4,4,RECOMMANDE,NaN,False,NaN,1,0,1,0,PLATINUM,...,93,9/2017,1000<10000,44.0,CHINA,-1,MODEL,BABY,1000<5000,WITHDRAWAL


In [4]:
data_train.shape

(100000, 23)

In [5]:
data_train.CLAIM_TYPE.value_counts()/len(data_train.CLAIM_TYPE)#ratio des classes en presence

-                           0.49977
NOT_RECEIVED                0.14808
SELLER_CANCEL_POSTERIORI    0.13782
WITHDRAWAL                  0.07001
DAMAGED                     0.05922
DIFFERENT                   0.04279
UNDEFINED                   0.04066
FAKE                        0.00165
Name: CLAIM_TYPE, dtype: float64

In [6]:
list(data_train)



['ID',
 'SHIPPING_MODE',
 'SHIPPING_PRICE',
 'WARRANTIES_FLG',
 'WARRANTIES_PRICE',
 'CARD_PAYMENT',
 'COUPON_PAYMENT',
 'RSP_PAYMENT',
 'WALLET_PAYMENT',
 'PRICECLUB_STATUS',
 'REGISTRATION_DATE',
 'PURCHASE_COUNT',
 'BUYER_BIRTHDAY_DATE',
 'BUYER_DEPARTMENT',
 'BUYING_DATE',
 'SELLER_SCORE_COUNT',
 'SELLER_SCORE_AVERAGE',
 'SELLER_COUNTRY',
 'SELLER_DEPARTMENT',
 'PRODUCT_TYPE',
 'PRODUCT_FAMILY',
 'ITEM_PRICE',
 'CLAIM_TYPE']

In [7]:
data_train.dtypes

ID                        int64
SHIPPING_MODE            object
SHIPPING_PRICE           object
WARRANTIES_FLG             bool
WARRANTIES_PRICE         object
CARD_PAYMENT              int64
COUPON_PAYMENT            int64
RSP_PAYMENT               int64
WALLET_PAYMENT            int64
PRICECLUB_STATUS         object
REGISTRATION_DATE         int64
PURCHASE_COUNT           object
BUYER_BIRTHDAY_DATE     float64
BUYER_DEPARTMENT          int64
BUYING_DATE              object
SELLER_SCORE_COUNT       object
SELLER_SCORE_AVERAGE    float64
SELLER_COUNTRY           object
SELLER_DEPARTMENT         int64
PRODUCT_TYPE             object
PRODUCT_FAMILY           object
ITEM_PRICE               object
CLAIM_TYPE               object
dtype: object

In [7]:
for i in list(data_train):
    print(i,data_train[i].unique())

ID [    0     1     2 ... 99997 99998 99999]
SHIPPING_MODE ['NORMAL' 'RECOMMANDE' 'EXPRESS_DELIVERY' 'SUIVI' 'SO_RECOMMANDE'
 'MONDIAL_RELAY' 'MONDIAL_RELAY_PREPAYE' 'SO_POINT_RELAIS' nan
 'CHRONOPOST' 'PICKUP' 'Kiala']
SHIPPING_PRICE [nan '5<10' '10<20' '<1' '1<5' '>20']
WARRANTIES_FLG [False  True]
WARRANTIES_PRICE [nan '5<20' '<5' '20<50' '50<100' '100<500']
CARD_PAYMENT [1 0]
COUPON_PAYMENT [0 1]
RSP_PAYMENT [1 0]
WALLET_PAYMENT [0 1]
PRICECLUB_STATUS ['UNSUBSCRIBED' 'PLATINUM' 'SILVER' 'REGULAR' 'GOLD' nan]
REGISTRATION_DATE [2015 2013 2007 2010 2005 2016 2004 2017 2012 2006 2009 2002 2014 2008
 2011 2003 2001]
PURCHASE_COUNT ['<5' '50<100' '>500' '5<20' '100<500' '20<50']
BUYER_BIRTHDAY_DATE [1992. 1952. 1991. 1955. 1984. 1987. 1986.   nan 1960. 1993. 1978. 1971.
 1990. 1983. 1973. 1985. 1988. 1994. 1998. 1970. 1956. 1961. 1974. 1989.
 1976. 1981. 1982. 1966. 1959. 2000. 1979. 1962. 1968. 1945. 1977. 1969.
 1980. 1954. 1957. 1997. 1995. 1996. 1958. 1972. 1946. 1963. 1948. 1965.
 

In [8]:
data_train_to_preprocess=data_train.drop(['ID','CLAIM_TYPE'],axis=1)


In [9]:
#on convertit y en entiers pour sklearn

le = preprocessing.LabelEncoder()
le.fit(data_train[['CLAIM_TYPE']])
data_train_y=data_train[['CLAIM_TYPE']].copy().apply(le.fit_transform)
data_train_y.head()


C:\Users\oussa\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,CLAIM_TYPE
0,1
1,0
2,4
3,0
4,7


In [10]:

le.inverse_transform(data_train_y.CLAIM_TYPE.values)

C:\Users\oussa\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['DAMAGED', '-', 'NOT_RECEIVED', ..., '-', 'WITHDRAWAL', '-'],
      dtype=object)

In [12]:
data_train_to_preprocess.head()

,SHIPPING_MODE,SHIPPING_PRICE,WARRANTIES_FLG,WARRANTIES_PRICE,CARD_PAYMENT,COUPON_PAYMENT,RSP_PAYMENT,WALLET_PAYMENT,PRICECLUB_STATUS,REGISTRATION_DATE,...,BUYER_BIRTHDAY_DATE,BUYER_DEPARTMENT,BUYING_DATE,SELLER_SCORE_COUNT,SELLER_SCORE_AVERAGE,SELLER_COUNTRY,SELLER_DEPARTMENT,PRODUCT_TYPE,PRODUCT_FAMILY,ITEM_PRICE
0,NORMAL,NaN,False,NaN,1,0,1,0,UNSUBSCRIBED,2015,...,1992.0,34,3/2017,10000<100000,46.0,"FRANCE, METROPOLITAN",61,CELLPHONE_ACCESSORY,ELECTRONICS,<10
1,NORMAL,NaN,False,NaN,1,0,0,0,UNSUBSCRIBED,2013,...,1952.0,77,8/2017,10000<100000,45.0,"FRANCE, METROPOLITAN",30,CELLPHONE_ACCESSORY,ELECTRONICS,<10
2,NORMAL,NaN,False,NaN,0,0,0,1,PLATINUM,2013,...,1991.0,58,5/2017,10000<100000,43.0,CHINA,-1,TOYS,BABY,<10
3,RECOMMANDE,NaN,True,5<20,1,0,0,0,UNSUBSCRIBED,2007,...,1955.0,31,5/2017,10000<100000,44.0,"FRANCE, METROPOLITAN",2,GARDEN_TOOLS,WHITE,50<100
4,RECOMMANDE,NaN,False,NaN,1,0,1,0,PLATINUM,2010,...,1984.0,93,9/2017,1000<10000,44.0,CHINA,-1,MODEL,BABY,1000<5000


In [13]:

#on convertir toutes les variables en variables categorielle, les na sont considerees comme des categories
data_train_to_preprocess.WARRANTIES_FLG=data_train_to_preprocess.WARRANTIES_FLG.astype(int)
data_train_to_preprocess.WARRANTIES_FLG=data_train_to_preprocess.WARRANTIES_FLG.astype(object)
data_train_to_preprocess.BUYER_DEPARTMENT=data_train_to_preprocess.BUYER_DEPARTMENT.astype(object)
data_train_to_preprocess.SELLER_DEPARTMENT=data_train_to_preprocess.SELLER_DEPARTMENT.astype(object)

data_train_to_preprocess.SELLER_SCORE_AVERAGE=data_train_to_preprocess.SELLER_SCORE_AVERAGE.astype(object)

data_train_to_preprocess.BUYER_BIRTHDAY_DATE=data_train_to_preprocess.BUYER_BIRTHDAY_DATE.astype(object)

data_train_to_preprocess.REGISTRATION_DATE=data_train_to_preprocess.REGISTRATION_DATE.astype(object)


data_train_to_preprocess=pd.get_dummies(data_train_to_preprocess,dummy_na=True)


#on rajooute des classes de covariables presentes dans le jeu de test (avec une quantite nulle) mais absente dans le jeu de train pour uniformite
data_train_to_preprocess['PRODUCT_TYPE_AUTOGRAPHES']=0
data_train_to_preprocess['BUYER_DEPARTMENT_100.0']=0
data_train_to_preprocess['PRODUCT_TYPE_INPUT ADAPTERS']=0
data_train_to_preprocess['PRODUCT_TYPE_ANSWERING MACHINE']=0

data_train_to_preprocess['SELLER_SCORE_AVERAGE_22.0']=0
data_train_to_preprocess['BUYER_BIRTHDAY_DATE_1909.0']=0
data_train_to_preprocess['BUYER_BIRTHDAY_DATE_1914.0']=0
data_train_to_preprocess['BUYER_BIRTHDAY_DATE_2008.0']=0





In [14]:
data_train_to_preprocess.head()

,CARD_PAYMENT,COUPON_PAYMENT,RSP_PAYMENT,WALLET_PAYMENT,SHIPPING_MODE_CHRONOPOST,SHIPPING_MODE_EXPRESS_DELIVERY,SHIPPING_MODE_Kiala,SHIPPING_MODE_MONDIAL_RELAY,SHIPPING_MODE_MONDIAL_RELAY_PREPAYE,SHIPPING_MODE_NORMAL,...,ITEM_PRICE_>5000,ITEM_PRICE_nan,PRODUCT_TYPE_AUTOGRAPHES,BUYER_DEPARTMENT_100.0,PRODUCT_TYPE_INPUT ADAPTERS,PRODUCT_TYPE_ANSWERING MACHINE,SELLER_SCORE_AVERAGE_22.0,BUYER_BIRTHDAY_DATE_1909.0,BUYER_BIRTHDAY_DATE_1914.0,BUYER_BIRTHDAY_DATE_2008.0
0,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
data_train=pd.concat([data_train['ID'],data_train_to_preprocess,data_train_y],axis=1)

In [16]:
list(data_train)

['ID',
 'CARD_PAYMENT',
 'COUPON_PAYMENT',
 'RSP_PAYMENT',
 'WALLET_PAYMENT',
 'SHIPPING_MODE_CHRONOPOST',
 'SHIPPING_MODE_EXPRESS_DELIVERY',
 'SHIPPING_MODE_Kiala',
 'SHIPPING_MODE_MONDIAL_RELAY',
 'SHIPPING_MODE_MONDIAL_RELAY_PREPAYE',
 'SHIPPING_MODE_NORMAL',
 'SHIPPING_MODE_PICKUP',
 'SHIPPING_MODE_RECOMMANDE',
 'SHIPPING_MODE_SO_POINT_RELAIS',
 'SHIPPING_MODE_SO_RECOMMANDE',
 'SHIPPING_MODE_SUIVI',
 'SHIPPING_MODE_nan',
 'SHIPPING_PRICE_10<20',
 'SHIPPING_PRICE_1<5',
 'SHIPPING_PRICE_5<10',
 'SHIPPING_PRICE_<1',
 'SHIPPING_PRICE_>20',
 'SHIPPING_PRICE_nan',
 'WARRANTIES_FLG_0.0',
 'WARRANTIES_FLG_1.0',
 'WARRANTIES_FLG_nan',
 'WARRANTIES_PRICE_100<500',
 'WARRANTIES_PRICE_20<50',
 'WARRANTIES_PRICE_50<100',
 'WARRANTIES_PRICE_5<20',
 'WARRANTIES_PRICE_<5',
 'WARRANTIES_PRICE_nan',
 'PRICECLUB_STATUS_GOLD',
 'PRICECLUB_STATUS_PLATINUM',
 'PRICECLUB_STATUS_REGULAR',
 'PRICECLUB_STATUS_SILVER',
 'PRICECLUB_STATUS_UNSUBSCRIBED',
 'PRICECLUB_STATUS_nan',
 'REGISTRATION_DATE_2001.0',
 '

In [17]:
data_train.shape


(100000, 629)

In [18]:
data_train.head()

In [43]:
print(data_train.shape)

(100000, 629)


In [53]:
list_X=list(data_train.drop(['CLAIM_TYPE'],axis=1))
list_X
#ATTTENTION X train et X test doivent avoir les features dans le meme ordre (sklearn convertit pandas en numpy array)

['ID',
 'CARD_PAYMENT',
 'COUPON_PAYMENT',
 'RSP_PAYMENT',
 'WALLET_PAYMENT',
 'SHIPPING_MODE_CHRONOPOST',
 'SHIPPING_MODE_EXPRESS_DELIVERY',
 'SHIPPING_MODE_Kiala',
 'SHIPPING_MODE_MONDIAL_RELAY',
 'SHIPPING_MODE_MONDIAL_RELAY_PREPAYE',
 'SHIPPING_MODE_NORMAL',
 'SHIPPING_MODE_PICKUP',
 'SHIPPING_MODE_RECOMMANDE',
 'SHIPPING_MODE_SO_POINT_RELAIS',
 'SHIPPING_MODE_SO_RECOMMANDE',
 'SHIPPING_MODE_SUIVI',
 'SHIPPING_MODE_nan',
 'SHIPPING_PRICE_10<20',
 'SHIPPING_PRICE_1<5',
 'SHIPPING_PRICE_5<10',
 'SHIPPING_PRICE_<1',
 'SHIPPING_PRICE_>20',
 'SHIPPING_PRICE_nan',
 'WARRANTIES_FLG_0.0',
 'WARRANTIES_FLG_1.0',
 'WARRANTIES_FLG_nan',
 'WARRANTIES_PRICE_100<500',
 'WARRANTIES_PRICE_20<50',
 'WARRANTIES_PRICE_50<100',
 'WARRANTIES_PRICE_5<20',
 'WARRANTIES_PRICE_<5',
 'WARRANTIES_PRICE_nan',
 'PRICECLUB_STATUS_GOLD',
 'PRICECLUB_STATUS_PLATINUM',
 'PRICECLUB_STATUS_REGULAR',
 'PRICECLUB_STATUS_SILVER',
 'PRICECLUB_STATUS_UNSUBSCRIBED',
 'PRICECLUB_STATUS_nan',
 'REGISTRATION_DATE_2001.0',
 '

In [27]:

X_train=data_train.drop(['ID','CLAIM_TYPE'],axis=1).values
y_train=data_train.CLAIM_TYPE.values

In [28]:
data_train.CLAIM_TYPE.value_counts()/len(data_train.CLAIM_TYPE)

0    0.49977
4    0.14808
5    0.13782
7    0.07001
1    0.05922
2    0.04279
6    0.04066
3    0.00165
Name: CLAIM_TYPE, dtype: float64

In [29]:
X_train.shape

(100000, 627)

In [62]:

#modele=RandomForestClassifier(n_estimators=250,max_features='sqrt',n_jobs=2,class_weight='balanced')
#modele=GradientBoostingClassifier()

modele=RandomForestClassifier(n_estimators=250,max_features='sqrt',n_jobs=2)

modele.fit(X_train,y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [63]:
#meme procedure de preparation pour le jeu de test
data_test=pd.read_csv(path+'input_test.csv')
data_test_to_preprocess=data_test.drop(['ID'],axis=1)


data_test_to_preprocess.WARRANTIES_FLG=data_test_to_preprocess.WARRANTIES_FLG.astype(int)
data_test_to_preprocess.WARRANTIES_FLG=data_test_to_preprocess.WARRANTIES_FLG.astype(object)




data_test_to_preprocess.SELLER_SCORE_AVERAGE=data_test_to_preprocess.SELLER_SCORE_AVERAGE.astype(object)

data_test_to_preprocess.BUYER_BIRTHDAY_DATE=data_test_to_preprocess.BUYER_BIRTHDAY_DATE.astype(object)

data_test_to_preprocess.REGISTRATION_DATE=data_test_to_preprocess.REGISTRATION_DATE.astype(object)




data_test_to_preprocess.BUYER_DEPARTMENT=data_test_to_preprocess.BUYER_DEPARTMENT.astype(object)
data_test_to_preprocess.SELLER_DEPARTMENT=data_test_to_preprocess.SELLER_DEPARTMENT.astype(object)
data_test_to_preprocess=pd.get_dummies(data_test_to_preprocess,dummy_na=True)


#on rajooute des classes de covariables presentes dans le jeu de train (avec une quantite nulle) mais absente dans le jeu de test pour uniformite
data_test_to_preprocess['SELLER_COUNTRY_LITHUANIA']=0
data_test_to_preprocess['ITEM_PRICE_>5000']=0
data_test_to_preprocess['PRODUCT_TYPE_MUSIC LOT']=0
data_test_to_preprocess['SELLER_COUNTRY_CYPRUS']=0
data_test_to_preprocess['SELLER_COUNTRY_GREECE']=0
data_test_to_preprocess['SELLER_COUNTRY_MALTA']=0




data_test_to_preprocess['BUYER_BIRTHDAY_DATE_1913.0']=0
data_test_to_preprocess['BUYER_BIRTHDAY_DATE_1916.0']=0

data_test_to_preprocess['BUYER_BIRTHDAY_DATE_1902.0']=0
data_test_to_preprocess['BUYER_BIRTHDAY_DATE_1923.0']=0



In [64]:
data_test=pd.concat([data_test['ID'],data_test_to_preprocess],axis=1)

In [65]:
data_test.shape

(99995, 628)

In [66]:
#ATTTENTION X train et X test doivent avoir les features dans le meme ordre (sklearn convertit pandas en numpy array)
data_test = data_test[list_X]



In [58]:
data_test.head()

,ID,CARD_PAYMENT,COUPON_PAYMENT,RSP_PAYMENT,WALLET_PAYMENT,SHIPPING_MODE_CHRONOPOST,SHIPPING_MODE_EXPRESS_DELIVERY,SHIPPING_MODE_Kiala,SHIPPING_MODE_MONDIAL_RELAY,SHIPPING_MODE_MONDIAL_RELAY_PREPAYE,...,ITEM_PRICE_>5000,ITEM_PRICE_nan,PRODUCT_TYPE_AUTOGRAPHES,BUYER_DEPARTMENT_100.0,PRODUCT_TYPE_INPUT ADAPTERS,PRODUCT_TYPE_ANSWERING MACHINE,SELLER_SCORE_AVERAGE_22.0,BUYER_BIRTHDAY_DATE_1909.0,BUYER_BIRTHDAY_DATE_1914.0,BUYER_BIRTHDAY_DATE_2008.0
0,100000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,100001,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100002,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,100003,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,100004,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:

data_train.drop(['CLAIM_TYPE'],axis=1).head()

,ID,CARD_PAYMENT,COUPON_PAYMENT,RSP_PAYMENT,WALLET_PAYMENT,SHIPPING_MODE_CHRONOPOST,SHIPPING_MODE_EXPRESS_DELIVERY,SHIPPING_MODE_Kiala,SHIPPING_MODE_MONDIAL_RELAY,SHIPPING_MODE_MONDIAL_RELAY_PREPAYE,...,ITEM_PRICE_>5000,ITEM_PRICE_nan,PRODUCT_TYPE_AUTOGRAPHES,BUYER_DEPARTMENT_100.0,PRODUCT_TYPE_INPUT ADAPTERS,PRODUCT_TYPE_ANSWERING MACHINE,SELLER_SCORE_AVERAGE_22.0,BUYER_BIRTHDAY_DATE_1909.0,BUYER_BIRTHDAY_DATE_1914.0,BUYER_BIRTHDAY_DATE_2008.0
0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [378]:
#comparaison des libelles des features train et test pour uniformite

list1=(list(data_train))
list2=(list(data_test))


print("Missing values in second list:", (set(list1).difference(list2)))

#print("Missing values in second list:", (set(list2).difference(list1)))


Missing values in second list: {'CLAIM_TYPE'}


In [67]:

X_test=data_test.drop(['ID'],axis=1).values
X_test.shape

(99995, 627)

In [68]:
y_test=modele.predict(X_test)

In [69]:
np.unique(y_test)

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [385]:
data_train.CLAIM_TYPE.unique()

array([1, 0, 4, 7, 6, 5, 2, 3], dtype=int64)

In [71]:
prediction=pd.DataFrame({'a':data_test.ID,'b':le.inverse_transform(y_test)})
prediction.columns = ['ID','CLAIM_TYPE']

C:\Users\oussa\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [73]:
prediction.CLAIM_TYPE.unique()

array(['SELLER_CANCEL_POSTERIORI', '-', 'DAMAGED', 'NOT_RECEIVED',
       'WITHDRAWAL', 'DIFFERENT', 'UNDEFINED', 'FAKE'], dtype=object)

In [57]:
data_train.CLAIM_TYPE.unique()

array([1, 0, 4, 7, 6, 5, 2, 3], dtype=int64)

In [74]:
prediction.to_csv('prediction_Rakuten_GB_All_Categorical_Corrected.csv',index=False,columns=['ID','CLAIM_TYPE'])

In [76]:
pd.read_csv('prediction_Rakuten_GB_All_Categorical_Corrected.csv')

,ID,CLAIM_TYPE
0,100000,SELLER_CANCEL_POSTERIORI
1,100001,-
2,100002,SELLER_CANCEL_POSTERIORI
3,100003,-
4,100004,DAMAGED
5,100005,-
6,100006,-
7,100007,-
8,100008,-
9,100009,NOT_RECEIVED
